In [1]:
import sys
sys.path.append('../')

In [2]:
"""
# Description: Consolidates all standardized and enriched datasets into a
#              point, line, and polygon feature class.
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
import os
import datetime
import arcpy
from scripts._1_assign_domains import AssignDomains
from scripts.utils import init_gdb, og_file_input 
from scripts._4_metadata_append import pt_metadata, ln_metadata, poly_metadata
from scripts._3_category import Category

workspace, scratch_workspace = init_gdb()

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216

# Reference
WFR_TF_Template = os.path.join(workspace,'a_Reference','WFR_TF_Template')


In [3]:
year = str(2023)

In [4]:
## POINTS
# INPUTS

nfpors_pt = og_file_input(prefix = 'nfpors_fuels_treatments_pts_enriched'+year, filetype = 'Point', gdb = os.path.join(workspace, 'c_Enriched'))
nfpors_fuels_treatments_pts_enriched = os.path.join(workspace,'c_Enriched', nfpors_pt)


pfirs_pt = og_file_input(prefix = 'PFIRS_enriched'+year, filetype = 'Point', gdb = os.path.join(workspace, 'c_Enriched'))
PFIRS_enriched = os.path.join(workspace,'c_Enriched', pfirs_pt)

cnra_pt = og_file_input(prefix = 'CNRA_enriched_pt'+year, filetype = 'Point', gdb = os.path.join(workspace, 'c_Enriched'))
CNRA_Enriched_pt = os.path.join(workspace,'c_Enriched',cnra_pt)

### TEMP: timber industry
timber_enriched_pt = []
for y in [2021, 2022, 2023]:
    timber_pt = og_file_input(prefix = 'Timber_Industry_enriched'+str(y), filetype = 'Point', gdb = os.path.join(workspace, 'c_Enriched'))
    timber_enriched_pt.append(os.path.join(workspace,'c_Enriched',timber_pt))


with arcpy.EnvManager(
        workspace=workspace,
        scratchWorkspace=scratch_workspace, 
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="xmin=-374900, ymin=-604500, xmax=540100, ymax=450000, spatial_reference='NAD 1983 California (Teale) Albers (Meters)'", 
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        transferDomains=False, 
        transferGDBAttributeProperties=False, 
        overwriteOutput = True,
):
    ### ITS_POINTS/LINES/POLYGONS
    points = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'d_Appended'), 
        out_name=f"Treat_n_harvests_points{year}_{date_id}", 
        geometry_type="POINT", 
        template=[WFR_TF_Template], 
        spatial_reference=3310, #"NAD 1983 California (Teale) Albers (Meters)"), 
        )

    point_append = arcpy.management.Append(
        inputs=[nfpors_fuels_treatments_pts_enriched, 
                CNRA_Enriched_pt, 
                PFIRS_enriched
               ] + timber_enriched_pt, 
        target=points, 
        schema_type="NO_TEST", 
        field_mapping="", 
        )

    AssignDomains(in_table=point_append)

    point_append_md = pt_metadata(point_append)    


      assigning domains
Point Metadata added successfully.


In [5]:
print("NFPORS: ", arcpy.management.GetCount(nfpors_fuels_treatments_pts_enriched)[0])
print("CNRA: ", arcpy.management.GetCount(CNRA_Enriched_pt)[0])
print("PFIRS: ", arcpy.management.GetCount(PFIRS_enriched)[0])
print("TIMBER: ", sum([int(arcpy.management.GetCount(d)[0]) for d in timber_enriched_pt]))
print("ALL: ", arcpy.management.GetCount(point_append)[0])

NFPORS:  185
CNRA:  2849
PFIRS:  12175
TIMBER:  72
ALL:  15281


In [6]:
##LINES
# Inputs

### TEMP: comment out PFIRS and CalTrans due to reproducibility issue
caltrans_ln = og_file_input(prefix = 'CalTrans_act_ln_enriched'+year, filetype = 'Line', gdb = os.path.join(workspace, 'c_Enriched'))
CalTrans_act_ln_enriched = os.path.join(workspace,'c_Enriched', caltrans_ln)

cnra_ln = og_file_input(prefix = 'CNRA_enriched_ln'+year, filetype = 'Line', gdb = os.path.join(workspace, 'c_Enriched'))
CNRA_Enriched_ln = os.path.join(workspace,'c_Enriched',cnra_ln)

with arcpy.EnvManager(
        workspace=workspace,
        scratchWorkspace=scratch_workspace, 
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="xmin=-374900, ymin=-604500, xmax=540100, ymax=450000, spatial_reference='NAD 1983 California (Teale) Albers (Meters)'", 
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        transferDomains=False, 
        transferGDBAttributeProperties=False, 
        overwriteOutput = True,
):
    
    lines = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'d_Appended'), 
        out_name=f"Treat_n_harvests_lines{year}_{date_id}", 
        geometry_type="POLYLINE", 
        template=[WFR_TF_Template], 
        spatial_reference=3310, #"NAD 1983 California (Teale) Albers (Meters)", 
    )

    lines_append = arcpy.management.Append(
        inputs=[CalTrans_act_ln_enriched, 
                CNRA_Enriched_ln
                ], 
        target=lines, 
        schema_type="NO_TEST", 
        field_mapping="", 

        )

    AssignDomains(in_table=lines_append)

    lines_append_md = ln_metadata(lines_append) 


      assigning domains
Line Metadata added successfully.


In [7]:
print("CalTrans: ", arcpy.management.GetCount(CalTrans_act_ln_enriched)[0])
print("CNRA: ", arcpy.management.GetCount(CNRA_Enriched_ln)[0])
print("ALL: ", arcpy.management.GetCount(lines_append)[0])

CalTrans:  103427
CNRA:  312
ALL:  103739


In [8]:
## POLYGON
# Inputs
blm_poly = og_file_input(prefix = 'BLM_enriched'+year, filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'))
BLM_enriched = os.path.join(workspace,'c_Enriched',blm_poly)

cnra_poly = og_file_input(prefix = 'CNRA_enriched_poly'+year, filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'))
CNRA_Enriched_poly = os.path.join(workspace,'c_Enriched',cnra_poly)

nfpors_poly = og_file_input(prefix = 'nfpors_fuels_treatments_polys_enriched'+year, filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'))
nfpors_fuels_treatments_enriched = os.path.join(workspace,'c_Enriched', nfpors_poly)

nps_poly = og_file_input(prefix = 'nps_flat_fuels_enriched'+year, filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'))
nps_flat_fuels_enriched = os.path.join(workspace,'c_Enriched',nps_poly)
"""
try:
    usfs_poly = og_file_input(prefix = 'usfs_edw_facts_common_attributes_enriched_'+year, filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'))
    usfs_edw_facts_common_attributes_enriched = [os.path.join(workspace,'c_Enriched',usfs_poly)]
except:
    if year!='2022':
        usfs_poly_04 = og_file_input(prefix = 'usfs_edw_facts_common_attributes_enriched', filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'), suffix="04_"+year)
        usfs_poly_05 = og_file_input(prefix = 'usfs_edw_facts_common_attributes_enriched', filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'), suffix="05_"+year)
        usfs_poly_06 = og_file_input(prefix = 'usfs_edw_facts_common_attributes_enriched', filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched'), suffix="06_"+year)
        usfs_edw_facts_common_attributes_enriched = [os.path.join(workspace,'c_Enriched',usfs_poly) for usfs_poly in [usfs_poly_04, usfs_poly_05, usfs_poly_06]]
    else:
        raise Exception("NO MATCHING DATA")
"""
usfs_prefix = "usfs_edw_facts_common_attributes_enriched_Actv_CommonAttribute_PL_Region{}_2023"

usfs_edw_facts_common_attributes_enriched = [os.path.join(workspace,
                                                          'c_Enriched',
                                                          og_file_input(
                                                              prefix = usfs_prefix.format(usfs_poly),
                                                              filetype = 'Polygon', gdb = os.path.join(workspace, 'c_Enriched')
                                                          )
                                                         ) for usfs_poly in ["04", "05", "06"]]
with arcpy.EnvManager(
        workspace=workspace,
        scratchWorkspace=scratch_workspace, 
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="xmin=-374900, ymin=-604500, xmax=540100, ymax=450000, spatial_reference='NAD 1983 California (Teale) Albers (Meters)'", 
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        transferDomains=False, 
        transferGDBAttributeProperties=False, 
        overwriteOutput = True,
):

    polygons = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'d_Appended'), 
        out_name=f"Treat_n_harvests_polygons{year}_{date_id}", 
        geometry_type="POLYGON", 
        template=[WFR_TF_Template], 
        spatial_reference=3310, #"NAD 1983 California (Teale) Albers (Meters)", 
        )

    polygons_append = arcpy.management.Append(
        inputs=[
                nfpors_fuels_treatments_enriched, 
                nps_flat_fuels_enriched, 
                CNRA_Enriched_poly, 
                BLM_enriched
                ] + usfs_edw_facts_common_attributes_enriched, # usfs 2023 is 3 seperate files
        target=polygons, 
        schema_type="NO_TEST", 
        field_mapping="", 
        )

    AssignDomains(in_table=polygons_append)

    polygons_append_md = poly_metadata(polygons_append) 


      assigning domains
Polygon Metadata added successfully.


In [9]:
print("NFPORS: ", arcpy.management.GetCount(nfpors_fuels_treatments_enriched)[0])
print("CNRA: ", arcpy.management.GetCount(CNRA_Enriched_poly)[0])
print("NPS: ", arcpy.management.GetCount(nps_flat_fuels_enriched)[0])
print("BLM: ", arcpy.management.GetCount(BLM_enriched)[0])
print("USFS: ", sum([int(arcpy.management.GetCount(d)[0]) for d in usfs_edw_facts_common_attributes_enriched]))
print("ALL: ", arcpy.management.GetCount(polygons_append)[0]) 

NFPORS:  661
CNRA:  24777
NPS:  160
BLM:  353
USFS:  31596
ALL:  57547


In [10]:
from scripts._3_category import Category

In [11]:
Category(point_append)
Category(lines_append)
Category(polygons_append)

      Calculate Activity Category Step 1/1
      Calculate Activity Category Step 1/1
      Calculate Activity Category Step 1/1


<Result 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\Interagency Tracking System.gdb\\d_Appended\\Treat_n_harvests_polygons2023_20240930'>